In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon

import geopandas as gpd

import folium

from pyproj import Transformer

In [2]:


# WGS84 (lat/lon) <-> Web Mercator (meters)
to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
to_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def project_to_3857(lon, lat):
    return to_3857.transform(lon, lat)

def project_to_4326(x, y):
    return to_4326.transform(x, y)

In [3]:
def bezier_curve(start, control1, control2, end, n=100):
    """Generate a cubic Bezier curve."""
    t = np.linspace(0, 1, n)[:, None]  # shape (n, 1)
    start, control1, control2, end = map(np.array, [start, control1, control2, end])
    curve = (
        (1 - t)**3 * start
        + 3 * (1 - t)**2 * t * control1
        + 3 * (1 - t) * t**2 * control2
        + t**3 * end
    )
    return curve


In [4]:
def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left


def flow_patch_geo(
    start_ll, end_ll,
    start_width,
    end_width,
    color,
    bend=0.1,
    curvature=50,
    inherit_normal=None,
    start_offset_m=0,
    end_offset_m=0,
    offset_direction="left",  # "left", "right", or np.array([x,y])

):
    """
    Construct a flow patch polygon where width and offsets are expressed in meters.

    start_offset_m / end_offset_m:
        Lateral shift (in meters) applied to the start/end points.
        Positive values shift to the chosen offset_direction.
    """

    # ---- Project lat/lon to EPSG:3857 meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector in meters
    d = end - start
    d_norm = np.linalg.norm(d)
    if d_norm == 0:
        raise ValueError("Start and end coordinates are identical; cannot compute direction.")

    ud = d / d_norm  # unit direction

    # Determine base normal
    if inherit_normal is None:
        n = perp(ud)
        n = n / np.linalg.norm(n)
    else:
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # ---- Determine offset direction ----
    if isinstance(offset_direction, str):
        if offset_direction == "left":
            offset_n = n
        elif offset_direction == "right":
            offset_n = -n
        else:
            raise ValueError("offset_direction must be 'left', 'right', or a 2D vector.")
    else:
        # custom vector
        offset_n = np.array(offset_direction)
        offset_n = offset_n / np.linalg.norm(offset_n)

    # ---- Apply lateral offsets in meters ----
    start = start + offset_n * start_offset_m
    end   = end   + offset_n * end_offset_m

    L = np.linalg.norm(d)
    C = curvature * L * 0.02   # final constant increases/decreases curve

    ctrl1 = start + 0.33 * d +  n * C
    ctrl2 = start + 0.66 * d -  n * C
    
    # ---- Construct edges with width in meters ----
    sw = start_width / 2.0
    ew = end_width   / 2.0

    top = bezier_curve(
        start + n * sw,
        ctrl1 + n * sw,
        ctrl2 + n * ew,
        end   + n * ew
    )

    bottom = bezier_curve(
        end   - n * ew,
        ctrl2 - n * ew,
        ctrl1 - n * sw,
        start - n * sw
    )

    # Combine and close
    verts = np.vstack([top, bottom, top[0]])

    # Convert back to lat/lon
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    width_and_color = [start_width, color]

    #return Polygon(verts_ll), n, width_and_color
    return Polygon(verts_ll), width_and_color

In [5]:
## list of lat longs 
'''
[(47.683729430644014, -122.34433918947462),
 (47.687259045590366, -122.34435249866571),
 (47.69108874059965, -122.34437859849116),
 (47.69480518468918, -122.34442577658399),
 (47.698157590581, -122.34452735842333),
 (47.70168220570689, -122.3445039810145),
 (47.70544935545624, -122.34455265309363),
 (47.71273742733618, -122.34473223501618),
 (47.7194691813143, -122.34484691221628),
 (47.723740055102255, -122.34489403611852)]

'''

'\n[(47.683729430644014, -122.34433918947462),\n (47.687259045590366, -122.34435249866571),\n (47.69108874059965, -122.34437859849116),\n (47.69480518468918, -122.34442577658399),\n (47.698157590581, -122.34452735842333),\n (47.70168220570689, -122.3445039810145),\n (47.70544935545624, -122.34455265309363),\n (47.71273742733618, -122.34473223501618),\n (47.7194691813143, -122.34484691221628),\n (47.723740055102255, -122.34489403611852)]\n\n'

In [6]:
## E line going from south to north
point_list = [
     (47.683729430644014, -122.34433918947462),
     (47.687259045590366, -122.34435249866571),
     (47.69108874059965, -122.34437859849116),
     (47.69480518468918, -122.34442577658399),
     (47.698157590581, -122.34452735842333),
     (47.70168220570689, -122.3445039810145),
     (47.70544935545624, -122.34455265309363),
     (47.71273742733618, -122.34473223501618),
     (47.7194691813143, -122.34484691221628),
     (47.723740055102255, -122.34489403611852)
]

rider_numbers = {
    0: [13, 0], #5
    1: [3, 2], #6
    2: [7, 4],  #4
    3: [2, 1],
    4: [3, 4],
    5: [6, 2],
    6: [5, 2],
    7: [2, 14],
    8: [2, 3]
}

In [7]:
#### make an ordered list that has the avg departing load at every stop

#print (type(rider_numbers))
avg_departing_load = []

current_load = 0
for stop_num, rider_nums in rider_numbers.items():
    current_load = current_load + rider_nums[0]
    current_load = current_load - rider_nums[1]
    avg_departing_load.append(current_load)
   # print (rider_nums)
   # print (current_load)

#print(avg_departing_load)

In [8]:
Starting_Y_Val = point_list[0][0]
Second_Y_Val = point_list[1][0]
Third_Y_Val = point_list[2][0]

Starting_X_Val = point_list[0][1]
Second_X_Val = point_list[1][1]
Third_X_Val = point_list[2][1]

In [9]:
WIDTH_MULTIPLIER = 10

polygon_list = []
width_and_color_list = []

In [10]:
def make_trunk_polygon(segment_num, width, end_offset = 0):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  # 40 meters
        end_width=width * WIDTH_MULTIPLIER,     # 40 meters
        start_offset_m = 0,
        end_offset_m = end_offset,
        bend = 0.1,
        curvature = 2,
        color = 'cornflowerblue'
    )     
    
    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [11]:
def make_boarding_polygon(segment_num, width, start_offset, end_offset):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  
        end_width=width * WIDTH_MULTIPLIER,   
        start_offset_m = start_offset,   
        end_offset_m = end_offset,
        bend = 0.1,
        curvature = 20,
        color = 'orange'
    )  

    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [12]:
def make_alighting_polygon(segment_num, width, start_offset, end_offset):
    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    X_Val_2 = point_list[segment_num + 1][1]
    Y_Val_2 = point_list[segment_num + 1][0]
    trunk, width_and_color = flow_patch_geo(
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        start_width=width * WIDTH_MULTIPLIER,  
        end_width=width * WIDTH_MULTIPLIER,   
        start_offset_m = start_offset,    ## start offset should be trunk_width * 0.5 + boarding_width * 0.5
        end_offset_m = end_offset,    ## end offset should be large (2.5 * end offset?)
        bend = 0.1,
        curvature = -20,
        color = 'green'
    )  

    polygon_list.append(trunk)
    width_and_color_list.append(width_and_color)

In [13]:
'''
def rectangle_patch_geo(
    start_ll,
    end_ll,
    width_m,
    inherit_normal=None,
    start_offset_m=0,
    end_offset_m=0,
    offset_direction="left",
):
    """
    Create a straight rectangle patch between start and end points.
    Width and offsets are in meters.
    Output is (Polygon, normal_vector) matching flow_patch_geo conventions.
    """

    # --- Project lon/lat → EPSG:3857 (meters) ---
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector
    d = end - start
    L = np.linalg.norm(d)
    if L == 0:
        raise ValueError("Start and end points are identical.")

    ud = d / L   # unit direction

    # --- Normal (perpendicular) vector ---
    if inherit_normal is None:
        n = np.array([-ud[1], ud[0]])
        n = n / np.linalg.norm(n)
    else:
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # --- Determine offset direction ---
    if isinstance(offset_direction, str):
        if offset_direction == "left":
            offset_n = n
        elif offset_direction == "right":
            offset_n = -n
        else:
            raise ValueError("offset_direction must be 'left', 'right', or a 2D vector.")
    else:
        offset_n = np.array(offset_direction)
        offset_n = offset_n / np.linalg.norm(offset_n)

    # --- Apply lateral offsets in meters ---
    start = start + offset_n * start_offset_m
    end   = end   + offset_n * end_offset_m

    # --- Build rectangle corners ---
    half_w = width_m / 2.0

    p1 = start + n * half_w
    p2 = end   + n * half_w
    p3 = end   - n * half_w
    p4 = start - n * half_w

    verts = [p1, p2, p3, p4, p1]

    # --- Convert back to lon/lat in (lon, lat) order for Folium ---
    verts_ll = []
    for x, y in verts:
        lat, lon = project_to_4326(x, y)
        verts_ll.append((lon, lat))   # Important: swap order!

    return Polygon(verts_ll), n

    '''

'\ndef rectangle_patch_geo(\n    start_ll,\n    end_ll,\n    width_m,\n    inherit_normal=None,\n    start_offset_m=0,\n    end_offset_m=0,\n    offset_direction="left",\n):\n    """\n    Create a straight rectangle patch between start and end points.\n    Width and offsets are in meters.\n    Output is (Polygon, normal_vector) matching flow_patch_geo conventions.\n    """\n\n    # --- Project lon/lat → EPSG:3857 (meters) ---\n    start = np.array(project_to_3857(*start_ll))\n    end   = np.array(project_to_3857(*end_ll))\n\n    # Direction vector\n    d = end - start\n    L = np.linalg.norm(d)\n    if L == 0:\n        raise ValueError("Start and end points are identical.")\n\n    ud = d / L   # unit direction\n\n    # --- Normal (perpendicular) vector ---\n    if inherit_normal is None:\n        n = np.array([-ud[1], ud[0]])\n        n = n / np.linalg.norm(n)\n    else:\n        n = inherit_normal / np.linalg.norm(inherit_normal)\n\n    # --- Determine offset direction ---\n    if isi

In [14]:


def rectangle_patch_geo(center_ll, width_m, height_m):
    """
    Create a simple axis-aligned rectangle centered on a given lon/lat point.
    Width and height are in meters.
    Returns (Polygon, up_vector).
    """

    # center_ll is (lat, lon)
    lat, lon = center_ll

    # BUT Web Mercator expects (lon, lat)
    cx, cy = project_to_3857(lon, lat)

    half_w = width_m / 2
    half_h = height_m / 2

    pts = [
        (cx - half_w, cy - half_h),
        (cx + half_w, cy - half_h),
        (cx + half_w, cy + half_h),
        (cx - half_w, cy + half_h),
        (cx - half_w, cy - half_h),
    ]

    print (pts)

    # Convert back to lon/lat → project_to_4326 returns (lat, lon)
    verts_ll = []
    for x, y in pts:
        lat2, lon2 = project_to_4326(x, y)
        verts_ll.append((lon2, lat2))   # Folium wants (lon, lat)

    return Polygon(verts_ll), np.array([0, 1])

    '''
def rectangle_patch_geo(center_ll, width_m, height_m):
    """
    Create a simple axis-aligned rectangle centered on a given lon/lat point.
    Width and height are in meters.
    Returns (Polygon, up_vector).
    """

    # Project center to EPSG:3857 (meters)
    cx, cy = project_to_3857(*center_ll)

    half_w = width_m / 2
    half_h = height_m / 2

    # Rectangle corners in meter space
    pts = [
        (cx - half_w, cy - half_h),
        (cx + half_w, cy - half_h),
        (cx + half_w, cy + half_h),
        (cx - half_w, cy + half_h),
        (cx - half_w, cy - half_h),
    ]

    print (pts)
    
    # Convert back to lon/lat, in (lon, lat) order for Folium
    verts_ll = []
    for x, y in pts:
        lat, lon = project_to_4326(x, y)
        verts_ll.append((lon, lat))

    # up_vector = (0, 1) in projected meters — included just for API consistency
    up_vector = np.array([0, 1])

    #return Polygon(verts_ll), up_vector
    return Polygon(verts_ll)'''

In [15]:
def make_rectangle_polygon(segment_num, width = 10):

    X_Val_1 = point_list[segment_num][1]
    Y_Val_1 = point_list[segment_num][0]
    #X_Val_2 = point_list[segment_num + 1][1]
    #Y_Val_2 = point_list[segment_num + 1][0]

    rect_attempt, width_and_color = rectangle_patch_geo(

    
        start_ll = (X_Val_1, Y_Val_1),
        end_ll = (X_Val_2, Y_Val_2),
        
        #start_ll = (Y_Val_1, X_Val_1),
        #end_ll = (Y_Val_2, X_Val_2),
        
        width_m = width * WIDTH_MULTIPLIER,  
        
        start_offset_m = 6,    ## start offset should be trunk_width * 0.5 + boarding_width * 0.5
        end_offset_m = 6,    ## end offset should be large (2.5 * end offset?)
        #color = 'grey'
    )  

    print ('rect_attempt - ', rect_attempt)
    
    polygon_list.append(rect_attempt)
    width_and_color_list.append([60, 'grey'])

In [16]:
print (width_and_color_list)

[]


In [17]:
for stop_num, rider_nums in rider_numbers.items():
    current_trunk_size = avg_departing_load[stop_num]
    current_boarders = rider_nums[0]
    current_alighters = rider_nums[1]
   # next_trunk_size = avg_departing_load[stop_num+1]
    
    ## calculate a center of width number so that the patches can all line up pretty :)
    ## planning to move the ending offset of the trunk and the boarding lines to line up correctly. 

    #next_stop_size = next_trunk_size + current_alighters
    
    #alighting_offset = current_trunk_size * 0.5 * WIDTH_MULTIPLIER + current_alighters * 0.5 * WIDTH_MULTIPLIER
    alighting_offset = (current_trunk_size + current_alighters) * 0.5

    #make_rectangle_polygon(stop_num)
    
    total_offset_trunk = 0
    if (stop_num < len(rider_numbers) - 1):
        next_board = rider_numbers[stop_num+1][0]
        next_alight = rider_numbers[stop_num+1][1]
        total_offset_trunk = -(next_board + next_alight) * 0.5


    #print ('total_offset_trunk = ', total_offset_trunk)
    if (stop_num == 0):
        
        print ('total_offset_trunk = ', total_offset_trunk, ' current_trunk_size = ', current_trunk_size)
        
        make_trunk_polygon(stop_num, current_trunk_size, total_offset_trunk * WIDTH_MULTIPLIER)

    
    else:
        boarding_offset = (previous_total_offset_trunk + (previous_trunk_size + current_boarders) * 0.5)
        make_trunk_polygon(stop_num, current_trunk_size, total_offset_trunk * WIDTH_MULTIPLIER)
        make_boarding_polygon(stop_num - 1, current_boarders, ((5 + boarding_offset) * 3 * WIDTH_MULTIPLIER), (boarding_offset * WIDTH_MULTIPLIER))
        make_alighting_polygon(stop_num, current_alighters, -(alighting_offset * WIDTH_MULTIPLIER), -((10 + alighting_offset) * 2 * WIDTH_MULTIPLIER))

    previous_trunk_size = current_trunk_size
    previous_total_offset_trunk = total_offset_trunk
    
    #print (rider_nums)

total_offset_trunk =  -2.5  current_trunk_size =  13


In [18]:
#colors = ['cornflowerblue', 'orange', 'cornflowerblue', 'green']


m = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=14)



for poly, width_and_color in zip(polygon_list, width_and_color_list):
    
    tt = "change this!"
    color = width_and_color[1]
    
    if(color == 'cornflowerblue'):
        tt = (f"Departing Load = {round(width_and_color[0] / WIDTH_MULTIPLIER)}") ## figure out how to get values from the method calls
    elif(color == 'orange'):
        tt = (f"Riders boarding = {round(width_and_color[0] / WIDTH_MULTIPLIER)}")
    elif(color == 'green'):
        tt = (f"Riders alighting = {round(width_and_color[0] / WIDTH_MULTIPLIER)}")
    elif(color == 'grey'):
        tt = ("in the stop zone!")
    folium.GeoJson(
        poly,
        style_function=lambda _, col = color: {
        #style_function=lambda _, col = : {
            "fillColor": col,
            "color": col,
            "weight": 1,
            "fillOpacity": 0.7
        }, 
        
        tooltip = tt
        #print ('tt -> ', tt)
    ).add_to(m)
    print ('tt -> ', tt)
m

tt ->  Departing Load = 13
tt ->  Departing Load = 14
tt ->  Riders boarding = 3
tt ->  Riders alighting = 2
tt ->  Departing Load = 17
tt ->  Riders boarding = 7
tt ->  Riders alighting = 4
tt ->  Departing Load = 18
tt ->  Riders boarding = 2
tt ->  Riders alighting = 1
tt ->  Departing Load = 17
tt ->  Riders boarding = 3
tt ->  Riders alighting = 4
tt ->  Departing Load = 21
tt ->  Riders boarding = 6
tt ->  Riders alighting = 2
tt ->  Departing Load = 24
tt ->  Riders boarding = 5
tt ->  Riders alighting = 2
tt ->  Departing Load = 12
tt ->  Riders boarding = 2
tt ->  Riders alighting = 14
tt ->  Departing Load = 11
tt ->  Riders boarding = 2
tt ->  Riders alighting = 3


In [19]:
from shapely.geometry import Polygon


def rectangle_center_geo(center_ll, width_m, height_m):
    """
    Create an axis-aligned rectangle centered on (lat, lon).
    Width and height are in meters.
    Returns a Polygon in (lon, lat) for Folium.
    """

    lat, lon = center_ll

    # Forward projection: (lon, lat)
    cx, cy = project_to_3857(lon, lat)

    hw = width_m / 2
    hh = height_m / 2

    # Rectangle corners in EPSG:3857 (meters)
    pts_3857 = [
        (cx - hw, cy - hh),
        (cx + hw, cy - hh),
        (cx + hw, cy + hh),
        (cx - hw, cy + hh),
        (cx - hw, cy - hh),
    ]

    # Convert back to lon/lat
    pts_ll = []
    for x, y in pts_3857:
        lon2, lat2 = project_to_4326(x, y)  # IMPORTANT: returns (lon, lat)
        pts_ll.append((lon2, lat2))         # Folium expects (lon, lat)

    return Polygon(pts_ll)

'''from shapely.geometry import Polygon

def rectangle_center_geo(center_ll, width_m, height_m):
    """
    Create an axis-aligned rectangle centered on a single (lat, lon) point.
    Width and height are in meters.
    Returns a Shapely Polygon in (lon, lat) coordinates.
    """

    lat, lon = center_ll

    # Convert center → meters
    cx, cy = project_to_3857(lon, lat)

    hw = width_m / 2
    hh = height_m / 2

    # Rectangle corners in 3857
    pts_3857 = [
        (cx - hw, cy - hh),
        (cx + hw, cy - hh),
        (cx + hw, cy + hh),
        (cx - hw, cy + hh),
        (cx - hw, cy - hh),
    ]

    # Convert corners back to lon/lat
    pts_ll = []
    for x, y in pts_3857:
        lat2, lon2 = project_to_4326(x, y)
        pts_ll.append((lon2, lat2))  # Folium expects (lon, lat)

    return Polygon(pts_ll)
    '''

'from shapely.geometry import Polygon\n\ndef rectangle_center_geo(center_ll, width_m, height_m):\n    """\n    Create an axis-aligned rectangle centered on a single (lat, lon) point.\n    Width and height are in meters.\n    Returns a Shapely Polygon in (lon, lat) coordinates.\n    """\n\n    lat, lon = center_ll\n\n    # Convert center → meters\n    cx, cy = project_to_3857(lon, lat)\n\n    hw = width_m / 2\n    hh = height_m / 2\n\n    # Rectangle corners in 3857\n    pts_3857 = [\n        (cx - hw, cy - hh),\n        (cx + hw, cy - hh),\n        (cx + hw, cy + hh),\n        (cx - hw, cy + hh),\n        (cx - hw, cy - hh),\n    ]\n\n    # Convert corners back to lon/lat\n    pts_ll = []\n    for x, y in pts_3857:\n        lat2, lon2 = project_to_4326(x, y)\n        pts_ll.append((lon2, lat2))  # Folium expects (lon, lat)\n\n    return Polygon(pts_ll)\n    '

In [20]:
## seattle - aurora -> [47.683729430644014, -122.34433918947462]

rect = rectangle_center_geo(
    #center_ll=(40.0, -73.9),
    center_ll=(47.68, -122.34),
   
    #center_ll=(47.683729430644014, -122.34433918947462),
    #center_ll=(-122.34433918947462, 47.683729430644014),
    width_m=20,
    height_m=10
)

#folium.GeoJson(rect.__geo_interface__).add_to(m)
folium.GeoJson(rect).add_to(m)

In [21]:
m